In [5]:
%tb
import json
from textwrap import dedent as d
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import numpy as np

app = dash.Dash(__name__)
app.css.append_css({'external_url': 'https://codepen.io/chriddyp/pen/dZVMbK.css'})

styles = {'pre': {'border': 'thin lightgrey solid', 'overflowX': 'scroll'}}

dots_pos = [(1,1),(3,1),(5,1),(7,1)]
f=4
fs = 1000
N = 10000
trace_t = [i/fs for i in range(N)]

def calc_envelope(dots):
    sorted_dots = dots.copy()
    sorted_dots.sort(key=lambda x: x[0])
    env = np.interp(trace_t, [p[0] for p in sorted_dots],
                             [p[1] for p in sorted_dots])
    return env

ref_i = np.array([np.cos(2*np.pi*f*t) for t in trace_t])
ref_q = np.array([np.sin(2*np.pi*f*t) for t in trace_t])

env = calc_envelope(dots_pos)
trace_i = ref_i * env
trace_q = ref_q * env

app.layout = html.Div(className='row', children=[
    dcc.Graph(
        id='basic-interactions',
        className='six columns',
        figure={
            'data': [{
                'x': trace_t,
                'y': trace_i,
                'name': 'I',
                'mode': 'lines'
            }, {
                'x': trace_t,
                'y': trace_q,
                'name': 'q',
                'mode': 'lines'
            }],
            'layout': {
                'shapes': [{
                    'type': 'circle',

                    'x0': -5,
                    'x1': 5,
                    'xref': 'x',
                    'xanchor': x,
                    'xsizemode':'pixel',

                    'y0': -5,
                    'y1': 5,
                    'yref': 'y',
                    'yanchor': y,
                    'ysizemode':'pixel',
                    'fillcolor': 'rgb(165, 10, 235)',
                    'line': {
                        'width': 4,
                        'color': 'rgb(165, 10, 235)'
                    }
                } for (x,y) in dots_pos]
            }
        },
        config={
            'editable': True,
            'edits': {
                'shapePosition': True
            }
        }
    ),
    html.Div(
        className='six columns',
        children=[
            html.Div(
                [
                    dcc.Markdown(
                        d("""
                **Zoom and Relayout Data**

            """)),
                    html.Pre(id='relayout-data', style=styles['pre']),
                ]
            )
        ]
    )
])

r = None
@app.callback(
    Output('basic-interactions', 'figure'),
    [Input('basic-interactions', 'relayoutData')],
    [State('basic-interactions', 'figure')])
def display_selected_data(relayoutData,fig):
    #return json.dumps(relayoutData, indent=2)
    if relayoutData:
        for i in range(len(dots_pos)):
            pos = [relayoutData.get(f'shapes[{i}].{coord}') for coord in ['xanchor', 'yanchor']]
            if pos[0]!=None:
                dots_pos[i] = (pos[0], (pos[1]))
    
    env = calc_envelope(dots_pos)
    trace_i = ref_i * env
    trace_q = ref_q * env
    #fig['layout']['shapes'].append({
    #                'type': 'vrect',
    #                'x0': 1,
    #                'x1': 2,
    #                'fillcolor': "LightSalmon",
    #                'opacity':0.5,
    #                'layer':"below",
    #                'line_width': 0
    #            })
    fig['data'][0]['x'] = trace_t
    fig['data'][0]['y'] = trace_i
    fig['data'][1]['x'] = trace_t
    fig['data'][1]['y'] = trace_q
    return fig
app.run_server(debug=False)

TypeError: can only concatenate str (not "int") to str

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jan/2021 19:59:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2021 19:59:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2021 19:59:06] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2021 19:59:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2021 19:59:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2021 19:59:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2021 19:59:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2021 19:59:11] "POST /_dash-update-component HTTP/1.1" 200 -


How should be interpolate between dots then?

Let's loop through each x value and do a straight line interpolation. Before first dot and after last dot are just constant amplitude.

In [77]:
sorted_dots = dots_pos.copy()
sorted_dots.sort(key=lambda x: x[0])
amp = np.interp(trace_t, [p[0] for p in sorted_dots],
                         [p[1] for p in sorted_dots])

In [82]:
dcc.Graph?

Init signature:
dcc.Graph(
    id=undefined,
    responsive=undefined,
    clickData=undefined,
    clickAnnotationData=undefined,
    hoverData=undefined,
    clear_on_unhover=undefined,
    selectedData=undefined,
    relayoutData=undefined,
    extendData=undefined,
    prependData=undefined,
    restyleData=undefined,
    figure=undefined,
    style=undefined,
    className=undefined,
    animate=undefined,
    animation_options=undefined,
    config=undefined,
    loading_state=undefined,
    **kwargs,
)
Docstring:     
A Graph component.
Graph can be used to render any plotly.js-powered data visualization.

You can define callbacks based on user interaction with Graphs such as
hovering, clicking or selecting

Keyword arguments:
- id (string; optional): The ID of this component, used to identify dash components
in callbacks. The ID needs to be unique across all of the
components in an app.
- responsive (a value equal to: true, false, 'auto'; default 'auto'): If True, the Plotly.js